# <u>__VAE__</u> - generate pseudo-random unit cells to augment our existing database

### <u>__27 February</u>:__  Making the mat files for some selected interesting looking topos from batch 8, though most of them look exactly the same as batch 7... 6, 5, 4... you get it.

### <u>__23 February</u>:__ Re-Thresholding the binary arrays in the 20FEB batch to 0.1...

### ... also, running another 200 topos, since the last batch was only 2,200 arrays

### <u>__20 February</u>:__ Making another 200 topos because I found that the fixed model is sooooooo much better at achieving targeted volume fractions.

### <u>__11 February</u>:__ Trained a new model where the KLD loss is much smaller and train/val track together, so now I'm creating 400 topos to see if they're noticeably different/more interesting... I think, mainly I'm looking for not having most of them be lattice, though in fairness, lattice is like 126 of 157 topos...

### [ <u>__25 December</u>:__ Generating the binary voxel plots of the big batch, batch4_23DEC ]

### <u>__24 December</u>:__ Generating topos using the ArchA model trained on 0.1-0.5 volume fractions. I determined that ArchB doesn't produce as wide a range of volume fractions as Arch A (on average like 4% vs 25%), 
###           so I also want to see how the $\Delta$ VolFrac is for a model trained on only 0.1-0.5 VolFrac

### <u>__22 December</u>:__ DOnno what happened below on 20DEC, but now I'm using... using Latin Hypercube sampling and a threshold of 0.5
##### Maybe that note should've been in the ArchB notebook? ... unclear

### <u>__20 December</u>:__ Starting over with a new batch... using Latin Hypercube sampling and a threshold of 0.5

### <u>__14 December</u>:__ Something went wonky with the first batch, as evidenced by the spherical interpolations plots not lining up with the voxel plots, so I'm starting over...
###    Also using a lower threshold - 0.6

###    (8:23pm:) Also running another batch with a different model, just to see if/how different it is... 
###    (21DEC:) I think the different model was the one without KLD annealing

### <u>__13 December</u>:__ Trained R-VAE to learn VolFrac, proved it works, time to generate data
###     Plotted voxel continuous and binary arrays, need to generate .MAT files



In [ ]:
import os

In [ ]:
batchno = '8'
date = '23FEB24'

# model_cp = 'vae_torch_enc16_64in_sigmoid_rebalanced_splits_11NOV.pth'
# model_cp = 'RVAE_torch_DataGen_VolFrac_RegrVAE_enc16_64in_64outerLyr_sigmoidBCE_rvae.pth'
model_cp = 'ModWghts_rVAE_test_ArchA_enc16_64in_64_meanKLDloss_largerbeta.pth'

# model_cp = 'Model_weights_DataGen_VolFrac_rVAE_ArchA_enc16_64in_64outerLyr_sigmoid_50pctVF_split_vsmall_KLD.pth'

# batchnote = 'LHS_50pctVF'
batchnote = 'LHS_newmodel'
sampling_type = 'LHS'
batch_folder = f'batch{batchno}_{date}_{batchnote}'


In [ ]:
import workflow_utils

from workflow_utils.FileDirectory import Directory

dirs =  Directory(rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2023/')
nbpath = dirs._10_VAE_regressor

In [ ]:
enc_filt_bl = 32
dec_filt_bl = 32
enc_dim = 16
outer_layer_dim=64

input_shape = (1, 64,64,64)
input_dim = input_shape[-1]

nbpath = os.path.join(dirs._10_VAE_regressor, 'rVAE_archA')
cp_dir = os.path.join(nbpath, 'model_CPs')

# 1 param - Volume Fraction
params = ['volFrac']
Y_file_suffix= 'VF' 
parlen = len(params)

In [ ]:
# archnote = f'_{outer_layer_dim}outerLyr_sigmoidBCE_KLannealing_non'
# archver = f'DataGen_VolFrac_RegrVAE_enc{enc_dim}_64in{archnote}'

In [ ]:
produced_path = os.path.join(nbpath, 'generated_data', batch_folder)
os.makedirs(produced_path, exist_ok=True)

In [ ]:
import pandas as pd
import numpy as np
import json
import glob
import os
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch

import torch.utils.data as data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
import torchsummary

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

from workflow_utils.DataPrep import Plot_Array

In [ ]:
# torch.cuda.set_device(1)

In [ ]:
class SamplingLayer(nn.Module):
    def forward(self, mean, log_var):
        epsilon = torch.randn_like(log_var)
        return mean + torch.exp(0.5 * log_var) * epsilon

In [ ]:
class RVAE(nn.Module):
    # def __init__(self, input_shape, latent_dim, ft_bank_baseline, dropout_alpha):
    def __init__(self, input_shape, enc_dim, enc_filt_bl, dec_filt_bl, outer_layer_dim):
    
        super(RVAE, self).__init__()
        # self.latent_dim = latent_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, outer_layer_dim, kernel_size=5, padding=0), 
            nn.ReLU(),

            nn.MaxPool3d(kernel_size=2),
            
            nn.Conv3d(outer_layer_dim, enc_filt_bl*2, kernel_size=5, padding=0),

            nn.ReLU(),
    
            nn.MaxPool3d(kernel_size=2),
    
            nn.Conv3d(enc_filt_bl*2, enc_filt_bl*2, kernel_size=5, padding=0),

            nn.ReLU(),
            
            
            nn.Flatten(start_dim=1, end_dim=-1),
    
            nn.Linear(enc_filt_bl*2*9*9*9, enc_dim*4),
            nn.ReLU(),
            nn.Linear(enc_dim*4, enc_dim*3),
            nn.ReLU()
        )
        

        self.z_mean = nn.Linear(enc_dim*3, enc_dim)
        self.z_log_var = nn.Linear(enc_dim*3, enc_dim)
        
        self.param_mean =  nn.Sequential(nn.Linear(enc_dim*3, enc_dim), 
                                     nn.ReLU(),
                                     nn.Linear(enc_dim, len(params))
                                    )
                                     
        self.param_log_var = nn.Sequential(nn.Linear(enc_dim*3, enc_dim),
                                       nn.ReLU(),
                                       nn.Linear(enc_dim, len(params))
                                      )
                                     
        
        self.sampling = SamplingLayer()
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(enc_dim+len(params), enc_dim*4),
            nn.ReLU(),
            nn.Linear(enc_dim*4, enc_dim*8),
            nn.ReLU(),
            nn.Linear(enc_dim*8, dec_filt_bl*2*9*9*9),
            nn.ReLU(),
            nn.Unflatten(1, (dec_filt_bl*2, 9,9,9)),
            
            
            nn.ConvTranspose3d(dec_filt_bl*2, dec_filt_bl*2, kernel_size=5, padding=0),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='trilinear'),   


            nn.ConvTranspose3d(dec_filt_bl*2, outer_layer_dim, kernel_size=5, padding=0),
            nn.ReLU(),
    
            nn.Upsample(scale_factor=2, mode='trilinear'), 
    
            nn.ConvTranspose3d(outer_layer_dim, 1, kernel_size=5, padding=0), 
            
            # nn.ReLU() # nn.ReLU(), # Removing, trying MAE loss... blurry could be sharpened...
            nn.Sigmoid()
        )
        # self.apply(self._init_weights)  # Apply custom weight initialization to decoder layers
    
    def forward(self, x):
        mean_log_var = self.encoder(x)
        z_mean = self.z_mean(mean_log_var)
        z_log_var = self.z_log_var(mean_log_var)
                                       
        z = self.sampling(z_mean, z_log_var)
        
        param_mean = self.param_mean(mean_log_var)
        param_log_var = self.param_log_var(mean_log_var)
        r = self.sampling(param_mean, param_log_var)
        
        zr = torch.cat((z,r), dim=1)
        
        # reconstructed = self.decoder(z)
        reconstructed = self.decoder(zr)
        
        
        return reconstructed, z_mean, z_log_var, r, param_mean, param_log_var

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv3d, nn.ConvTranspose3d, nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


### <u>__Model parameters__</u>

In [ ]:
# Create the VAE model

rvae = torch.nn.DataParallel(RVAE(input_shape, enc_dim, enc_filt_bl, dec_filt_bl,  outer_layer_dim)).to(device)

In [ ]:
best_weights_path = os.path.join(cp_dir, model_cp)
# best_weights_path = os.path.join(cp_dir, 'vae_torch_vae_enc16_64in_relu.pth')
print(best_weights_path)


In [ ]:
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict

In [ ]:
""" 
This is for taking a state_dict that was created by a non-nn.DataParallel process and turning it into one with a nn.DataParallel structure by adding
'module.' to every key
"""
non_data_parallel_pth = False

if non_data_parallel_pth:
    non_data_parallel_weights = torch.load(best_weights_path)

    # original saved file with DataParallel
    data_parallel_weights = OrderedDict()
    for k, v in non_data_parallel_weights.items():
        name = 'module.'+k # remove `module.`
        data_parallel_weights[name] = v
    # # load params
    # model.load_state_dict(new_state_dict)
else:
    data_parallel_weights = torch.load(best_weights_path)

In [ ]:
rvae.load_state_dict(data_parallel_weights)

In [ ]:
# print(rvae)

# rvae_params = list(rvae.parameters())

# for i in rvae_params:
#     print(i.shape)

# Generate Unit Cells

In [ ]:
# vae.load_state_dict(torch.load(best_weights_path))

In [ ]:
rvae.eval()

In [ ]:
decoder = rvae.module.decoder # for DataParallel
# decoder = vae.decoder

In [ ]:
decoder.cuda()

### __Data Generation__

### __20FEB__: generating 200 topos... see top of notebook

### __11FEB__: generating 400 topos with new model where the KLD loss is better and the beta scaling factor is less small, i.e., 0.01 instead of 0.0001

### __25DEC__: generating the binary voxel plots of Batch 4

### __24DEC__: generating 300 topos with ... the usual... but with the model trained on 50% volfrac

#### __23DEC__: generating 250 topos with 23 intervals of volFrac to see if ArchA is better at linking VolFrac to topo

#### __13DEC__: Generating ~~20?~~~ 23 intervals of volume fractions

#### __14DEC__: Generating 23 intervals of volume fractions... new vectors, 400 topos

#### __...8:23pm__: Generating 23 intervals of volume fractions... new vectors, 400 topos... using a different model

In [ ]:
threshold = True

In [ ]:
import nltk
import re
import random
from nltk.corpus import wordnet as wn

# nltk.download('words')
# nltk.download('wordnet')

In [ ]:
# Function to get a random synset word without hyphens or underscores

def get_random_word(pos):
    while True:
        synsets = list(wn.all_synsets(pos))
        random_synset = random.choice(synsets)
        word = random_synset.lemmas()[0].name()
        if not re.search(r'[-_/]', word) and len(word) <= 8:
            return word.capitalize()

In [ ]:
num_topos = 200

# Generate unique topology names (adjective + noun) for each vector
topo_names = []
for _ in range(num_topos):
    adj = get_random_word("a")  # Adjective
    noun = get_random_word("n")  # Noun
    UC_topo_name = adj + noun
    topo_names.append(UC_topo_name)

In [ ]:
# with open(f'{produced_path}/batch4exprt_LHS_23DEC_topo_names.txt' ,'w') as f:
#     for item in topo_names:
#         f.write(item + "\n")
# f.close()

In [ ]:
# volfracs = np.round(np.arange(0.1,1.0,0.04), 2)
# volfracs = np.round(np.arange(0.1,0.51,0.04), 2)
volfracs = np.round(np.arange(0.1,0.61,0.04), 2)
volfracs

In [ ]:
# volfracs

In [ ]:
# for vf in volfracs:
#     print(np.round(vf*100,1))
#     print(vf)
#     print(vf*100)

In [ ]:
from scipy.stats.qmc import LatinHypercube as LHS
from scipy.stats import norm

LHsampler = LHS(d=16)

lhs_array = LHsampler.random(n=num_topos)

vector_array = lhs_norm_array = norm.ppf(lhs_array)

In [ ]:
print(f'{produced_path}/{batch_folder}_LHSvectors.npy')

In [ ]:
np.save(f'{produced_path}/{batch_folder}_LHSvectors.npy', lhs_norm_array)

In [ ]:
# vector_array = np.load(f'{produced_path}/{batch_folder}_LHSvectors.npy')

In [ ]:
vectors = []
for i in range(vector_array.shape[0]):
    vec = np.expand_dims(vector_array[i,:], 0)
    vec = torch.from_numpy(vec).to(dtype=torch.float32)
    vectors.append(vec)

In [ ]:
# vectors = [torch.randn(1, 16) for _ in range(num_topos)] #Old way


# Create a dictionary with vectors and their corresponding part names
vectordic = {tname: vector for tname, vector in zip(topo_names, vectors)}

In [ ]:
gen_data_dic = {}

In [ ]:
convert_to_numpy = False

# threshold = 0.75
threshold = 0.1


for topo, vector in vectordic.items():
    gen_data_dic[topo] = {}
    gen_data_dic[topo]['base vector'] = vector
    
    for num, vf in enumerate(volfracs):
        voxdic = {}
        vfrnd = np.round(vf*100,1)
        vf_entry = 'VF'+f'{vfrnd:.0f}'+'%'
        # voxdic[vf_entry] = {}
        
        # Modify the base vector for volume fraction
        vf = vf.astype(np.float32)
        vf_tensor = torch.tensor([[vf]])
        # vf_tensor = torch.tensor([[vf]]).astype(float32)
        vec = torch.cat((vector, vf_tensor), dim=1)
        # voxdic[vf_entry]['modified vector'] = vec

        # Decode the vector into the topology voxel array
        dec_array = np.squeeze(decoder(vec.cuda()).detach().cpu().numpy(), axis=(0,1))
        
        # voxdic[vf_entry]['decoded array'] = dec_array
        
        # Convert the array to binary based on a threshold
        binary_array = (dec_array >= threshold).astype(int)
        
        # voxdic[vf_entry]['binary array'] = binary_array
        
        # Compute the volume fraction of the binary array
        volfrac = (np.sum(binary_array == 1)) / (64**3)
        
        # Format the volume fraction
        volfrac_pct = str(np.round(volfrac,3) * 100)+' %'
        volfrac_str = f'{volfrac:.5f}'
        

        
        voxdic = {'Target volume fraction': vf,
                  'Modified vector': vec,
                  'Sampling method': sampling_type,
                  'decoded array': dec_array,
                  'binary array 0.1 threshold': binary_array,
                  'Actual volume fraction': {'percent': volfrac_pct,
                                     'decimal': volfrac,
                                     'string':  volfrac_str},
                 }
        
        gen_data_dic[topo][vf_entry] = voxdic


In [ ]:
volfrac_df = pd.DataFrame(columns = ['topo', 'volfrac delta'])

In [ ]:
topo_list = []
vf_range_list = []

loaded_dict = gen_data_dic

for key in list(loaded_dict.keys()):
    # print(key)
    topo_list.append(key)
    vfkeys = list(loaded_dict[key].keys())[1:]
    highvf = vfkeys[-1]
    lowvf = vfkeys[0]
    vf_range = loaded_dict[key][highvf]['Actual volume fraction']['decimal'] -  loaded_dict[key][lowvf]['Actual volume fraction']['decimal']
    
    vf_range_list.append(vf_range)
    # for subkey in list(loaded_dict[key].keys())[1:]:
    #     print('Target, Actual:', '\t', loaded_dict[key][subkey]['Target volume fraction'], '\t', loaded_dict[key][subkey]['Actual volume fraction']['percent'])

volfrac_df['topo'] = topo_list
volfrac_df['volfrac delta'] = vf_range_list

In [ ]:
volfrac_df['volfrac delta pct'] = volfrac_df['volfrac delta']*100

In [ ]:
(volfrac_df['volfrac delta pct'] > 45).value_counts()

In [ ]:
# for key in list(loaded_dict.keys())[:250]:
#     print(key)
#     vfkeys = list(loaded_dict[key].keys())[1:]
#     highvf = vfkeys[-1]
#     lowvf = vfkeys[0]
# #     print('Target, Actual:', '\t', loaded_dict[key][lowvf]['Target volume fraction'], '\t', loaded_dict[key][lowvf]['Actual volume fraction']['percent'])
# #     print('Target, Actual:', '\t', loaded_dict[key][highvf]['Target volume fraction'], '\t', loaded_dict[key][highvf]['Actual volume fraction']['percent'])
    
#     print('Low VF:', '\t', loaded_dict[key][lowvf]['Actual volume fraction']['percent'])
#     print('High VF:', '\t', loaded_dict[key][highvf]['Actual volume fraction']['percent'])
    

In [ ]:
volfrac_df['volfrac delta pct'].describe()

In [ ]:
# arrdict.keys()

In [ ]:
# arrdict_path = os.path.join(produced_path, synthcells_batch{batchno}_dict_{date}.json')

In [ ]:
# # ChatGPT to the rescue! for Torch.Tensor is non-JSON serializable
# # Narp, takes toooooo long

# import json
# import numpy as np
# import torch

# def convert_to_serializable(data):
#     if isinstance(data, torch.Tensor):
#         return {"torch_tensor": data.tolist()}
#     elif isinstance(data, np.ndarray):
#         return {"numpy_array": data.tolist()}
#     elif isinstance(data, np.float64):
#         return {"numpy_float": float(data)}
#     elif isinstance(data, dict):
#         return {key: convert_to_serializable(value) for key, value in data.items()}
#     else:
#         return data

# def convert_to_original_type(data):
#     if isinstance(data, dict):
#         if "torch_tensor" in data:
#             return torch.tensor(data["torch_tensor"])
#         elif "numpy_array" in data:
#             return np.array(data["numpy_array"], dtype=np.float64)
#         elif "numpy_float" in data:
#             return np.float64(data["numpy_float"])
#         else:
#             return {key: convert_to_original_type(value) for key, value in data.items()}
#     else:
#         return data

# def save_dict_to_json(file_path, data_dict):
#     serializable_dict = {key: convert_to_serializable(value) for key, value in data_dict.items()}
#     with open(file_path, 'w') as json_file:
#         json.dump(serializable_dict, json_file)

# def load_dict_from_json(file_path):
#     with open(file_path, 'r') as json_file:
#         serializable_dict = json.load(json_file)
#     original_dict = {key: convert_to_original_type(value) for key, value in serializable_dict.items()}
#     return original_dict

In [ ]:
# # Save the dictionary to a JSON file
# save_dict_to_json(arrdict_path, arrdict)

In [ ]:
# # Load the JSON file and convert the values back to their original formats
# loaded_dict = load_dict_from_json(arrdict_path)

In [ ]:
gendatadic_pkl_path = os.path.join(nbpath, f'generated_data/{batch_folder}/synthcells_batch{batchno}_dict_betterKLD_model_{date}.pkl')

gendatadic_pkl_path

In [ ]:
# This works in a reasonable amount of time, thanks again ChatGPT

import pickle

def save_dict_to_pickle(file_path, data_dict):
    with open(file_path, 'wb') as pickle_file:
        pickle.dump(data_dict, pickle_file)

def load_dict_from_pickle(file_path):
    with open(file_path, 'rb') as pickle_file:
        loaded_dict = pickle.load(pickle_file)
    return loaded_dict

#### Single dict saving and loading

In [ ]:
# Save the dictionary to a pickle file
save_dict_to_pickle(gendatadic_pkl_path, gen_data_dic)

In [ ]:
# Load the dictionary from the pickle file
loaded_dict = load_dict_from_pickle(gendatadic_pkl_path)

### Re-thresholding binary arrays to 0.1

In [ ]:
# loaded_dict.keys()

In [ ]:
# plot_array_range(loaded_dict['CariousLamarck']['VF10%']['binary array'], export_png=False, show=True, binary=True)

In [ ]:
# plot_array_range(loaded_dict['CariousLamarck']['VF10%']['binary array 0.1 threshold'], export_png=False, show=True, binary=True)

In [ ]:
# threshold = 0.1

In [ ]:
# for topo in list(loaded_dict.keys())[:]:
    
    topodict = loaded_dict[topo]
    
    for volfrac in list(topodict.keys())[1:]:
        
        vfdict = topodict[volfrac]
        
        cont_array = vfdict['decoded array']
        
        bin_array_rethresh = (cont_array >= threshold).astype(int)
        
        rethr_vfrac = (np.sum(bin_array_rethresh == 1)) / (64**3)
        rethr_volfrac_pct = str(np.round(rethr_vfrac,3) * 100)+' %'
        rethr_volfrac_str = f'{rethr_vfrac:.5f}'
        
        rethr_volfracs = {'percent': rethr_volfrac_pct,
                                     'decimal': rethr_vfrac,
                                     'string':  rethr_volfrac_str}
        
       
        loaded_dict[topo][volfrac]['Rethresholded volume fraction'] = rethr_volfracs
        
        loaded_dict[topo][volfrac]['binary array 0.1 threshold'] = bin_array_rethresh

In [ ]:
# gendatadic_pkl_path

In [ ]:
# Save/overwrite

# save_dict_to_pickle(gendatadic_pkl_path, loaded_dict)

In [ ]:
# print(loaded_dict['LifelessMaoism'].keys(),

# loaded_dict['LifelessMaoism']['VF10%'].keys(),

# loaded_dict['LifelessMaoism']['VF10%']['Actual volume fraction']['percent'],

# loaded_dict['LifelessMaoism']['VF10%']['Rethresholded volume fraction']['percent'],)

#### In case of splitting the dict into two halves to save it... needed to do this with 800 topos

In [ ]:
# """
# Saving the generated data as separate pkl files because with 800 topos the kernel died...
# """

# dic_len = len(topo_names)
# dic_half = dic_len//2
# keys = list(gen_data_dic.keys())


# gendatadic_keys_firsthalf = {key: gen_data_dic[key] for key in keys[:dic_half]}
# gendatadic_keys_secondhalf = {key: gen_data_dic[key] for key in keys[dic_half:]}



In [ ]:
# gendatadic_firsthalf_pkl_path = os.path.join(nbpath, f'generated_data/{batch_folder}/synthcells_batch{batchno}_800{sampling_type}_dict_{date}_firsthalf.pkl')
# gendatadic_secondhalf_pkl_path = os.path.join(nbpath, f'generated_data/{batch_folder}/synthcells_batch{batchno}_800{sampling_type}_dict_{date}_secondhalf.pkl')
# print(gendatadic_firsthalf_pkl_path)
# print(gendatadic_secondhalf_pkl_path)

In [ ]:
# Save the dictionary to a pickle file
# save_dict_to_pickle(gendatadic_firsthalf_pkl_path, gendatadic_keys_firsthalf)

In [ ]:
# save_dict_to_pickle(gendatadic_secondhalf_pkl_path, gendatadic_keys_secondhalf)

In [ ]:

load_first_half = load_dict_from_pickle(gendatadic_firsthalf_pkl_path)

len(list(load_first_half.keys()))

load_second_half = load_dict_from_pickle(gendatadic_secondhalf_pkl_path)

loaded_dict = {**load_first_half, **load_second_half}

print(len(list(load_first_half.keys())), len(list(load_second_half.keys())), len(list(loaded_dict.keys())))

del load_first_half, load_second_half

In [ ]:
mat_path = os.path.join(produced_path, 'MAT_files')
os.makedirs(mat_path, exist_ok=True)
cont_plot_path = os.path.join(produced_path, 'continuous_voxel_plots')
os.makedirs(cont_plot_path, exist_ok=True)
bin_plot_path = os.path.join(produced_path, 'binary_voxel_plots')
os.makedirs(bin_plot_path, exist_ok=True)

In [ ]:
print(mat_path)

In [ ]:
# gen_data_dic.keys()

In [ ]:
# topo = 'DarkAbel'
# for key in list(gen_data_dic[topo].keys())[1:]:
#     print(f"target VF: {key[2:]}, actual VF: {gen_data_dic[topo][key]['Actual volume fraction']['percent']}")

In [ ]:
# plot_array_range(loaded_dict['ActinoidMedicaid']['VF38%']['binary array'], show=True, export_png=False, binary=True, x_range=(0,32))

In [ ]:
loaded_dict['LifelessMaoism']['VF10%'].keys()


In [ ]:
len(list(loaded_dict.keys()))*23//2

In [ ]:
len(list(loaded_dict.keys()))*23

In [ ]:
# half = len(list(loaded_dict.keys()))*23//2

unit_count=0


# Save as .mat files

from scipy.io import savemat

In [ ]:
topodict.keys()

In [ ]:
# 27 FEB - a few selected interesting topos from batch 8
selected_topos = ['FeverishJeth', 'OverbusyCellini', 'InnocentAugend', 'DatelessInfamy','EruditeSteel','FootsoreNetting','RingedMyrtle', 'RustlessMarimba']

In [ ]:
gen_data_dic = loaded_dict # !!!!! For loaded_dict that was rethresholded

# To export to MATLAB files
# for key in list(gen_data_dic.keys())[:]:
for key in selected_topos:

    topodict = gen_data_dic[key]

    for subkey in list(topodict.keys())[1:]:
        vfincr_dict = topodict[subkey]
        
        # increm = subdict['sub_increment']
        volfrac = topodict[subkey]['Actual volume fraction']['decimal']
        # volfrac = topodict[subkey]['Rethresholded volume fraction']['decimal'] # !!!!!
        
        volfrac_formatted = f'{volfrac:.2e}'
        
        # fullVF = subdict['volFrac']
        # volfrac = f'{fullVF:.3f}'
        name = f'synth{sampling_type}_{key}_{volfrac_formatted}_{subkey[2:4]}VFtgt'
        
        matfile_dict = {'name': name,
                        'volFrac': volfrac,
                        'binary_voxel_array': vfincr_dict['binary array 0.1 threshold']}

        # if unit_count <= half:
            # filepath = os.path.join(mat_path, '0_half', name + '.mat')
        # elif unit_count > half:
            # filepath = os.path.join(mat_path, 'half_end', name + '.mat')
            
        filepath = os.path.join(mat_path, name + '.mat')
            
            
        # print(filepath)
        savemat(filepath, matfile_dict)
        
        unit_count += 1

In [ ]:
count

In [ ]:
count2

In [ ]:
unit_count

In [ ]:
len(list(gen_data_dic.keys()))*23//2

In [ ]:
unit_count

In [ ]:
len(list(loaded_dict.keys()))

In [ ]:
len(gen_data_dic.keys())

In [ ]:
# for key in gen_data_dic.keys():
#     print(len(gen_data_dic[key].keys()))

In [ ]:
# loadmat('/data/tigusa1/MLTO_UCAH/MLTO_2023/0_data/source_MAT_files/topopt/final_struc_4A_75iso_4.2e-01.mat')

In [ ]:
# loadmat(os.path.join(mat_path, 'synth_GeorgianSermon_5.84e-01.mat'))

In [ ]:
import plotly.express as px

def plot_array_range(array, colorscale='sunset', width=800, height=800, marker_scaler=8, x_range=None, y_range=None, z_range=None, 
                                plotpath = None, show=False, export_png=True, export_html=False, binary=False):
    z, x, y = array.nonzero()
    
    # Filter data based on specified ranges
    if x_range is not None:
        x_mask = (x >= x_range[0]) & (x <= x_range[1])
        x = x[x_mask]
        y = y[x_mask]
        z = z[x_mask]
        
    if y_range is not None:
        y_mask = (y >= y_range[0]) & (y <= y_range[1])
        x = x[y_mask]
        y = y[y_mask]
        z = z[y_mask]
        
    if z_range is not None:
        z_mask = (z >= z_range[0]) & (z <= z_range[1])
        x = x[z_mask]
        y = y[z_mask]
        z = z[z_mask]

    
    arr_plot_df = pd.DataFrame({'x': x, 'y': y, 'z': z})
    

    if binary:
        p = px.scatter_3d(x=arr_plot_df['x'], y=arr_plot_df['y'], z=arr_plot_df['z'], width=width, height=height)
        p.update_traces(marker=dict(size=12, line=dict(width=0.1, color='black')))
        p.update_traces(marker_size=3)                                        
        
    else:
        value_list = []
        for i in range(len(x)):
            value_list.append(array[z[i], x[i], y[i]])

        arr_plot_df["activation_value"] = value_list
        arr_plot_df['markersize_scaler'] = (arr_plot_df["activation_value"] - arr_plot_df["activation_value"].min()) / (arr_plot_df["activation_value"].max() - arr_plot_df["activation_value"].min())

        p = px.scatter_3d(x=arr_plot_df['x'], 
                          y=arr_plot_df['y'], 
                          z=arr_plot_df['z'], 
                          color=arr_plot_df['activation_value'], 
                          opacity=1.0,
                          color_continuous_scale=colorscale,
                          width=width,
                          height=height)

        p.update_traces(marker=dict(size=12, line=dict(width=0.1, color='grey')))
        p.update_traces(marker_size=arr_plot_df['markersize_scaler'] * marker_scaler)
    
        
    
    if show:
        p.show()
    else:
        pass
    
    if export_png:
        p.write_image(plotpath+'.png')
    else:
        pass
    
    if export_html:
        p.write_image(plotpath+'.html')
    else:
        pass
    del p, arr_plot_df
    


In [ ]:
# gen_data_dic['DarkAbel']['VF38%']['

In [ ]:
# For loaded dictionary
# gen_data_dic = loaded_dict

In [ ]:
topo_keys = []
for file in os.listdir(cont_plot_path):
    topo_name = file.split('_')[0]
    topo_keys.append(topo_name)
topo_key_set = set(topo_keys)

## __20 FEB__: Stopped @ CreakyHinault, number 37

In [ ]:
for i, key in enumerate(gen_data_dic.keys()):
    print(i, key)

In [ ]:
print(len(list(gen_data_dic.keys())))

In [ ]:
gen_data_dic = loaded_dict

In [ ]:
vfkey = 'VF38%'

plot_range = (32, 64)

# count = 0

# For plotting continuous voxel arrays

for key in list(gen_data_dic.keys())[:]:
    if key in topo_key_set:
        # count +=1
        pass
    else:
        # count +=1
        # pass
        vecdict = gen_data_dic[key]

        topo_name = key

        array = gen_data_dic[key][vfkey]['decoded array']

        plot_path = cont_plot_path


        for incr in plot_range:
            if incr==32:
                plotincr = 'half'
            else:
                plotincr = 'full'
            filename = f'{key}_{vfkey[:-1]}_ref_{plotincr}'

            plot_filepath = os.path.join(plot_path, filename)
            plot_array_range(array, x_range=(0,incr), plotpath = plot_filepath, binary=False)


In [ ]:
binary_plot_path = os.path.join(produced_path, 'binary_voxel_plots')

In [ ]:
binary_plot_path

In [ ]:
# For plotting binary voxel arrays
vfkey = 'VF38%'

# If the dict was loaded from a saved pkl file
gen_data_dic = loaded_dict

for key in list(gen_data_dic.keys())[:]:
    vecdict = gen_data_dic[key]
    
    topo_name = key
    
    array = gen_data_dic[key][vfkey]['binary array']
    actual_vf = gen_data_dic[key][vfkey]['Actual volume fraction']['percent'][:4]
    
    plot_path = bin_plot_path
    
    plot_range = (32, 64)
    
    for incr in plot_range:
        if incr==32:
            plotincr = 'half'
        else:
            plotincr = 'full'
        filename = f'{key}_{vfkey[:-1]}_ref_{plotincr}_BinVox_VF{actual_vf}'
        
        plot_filepath = os.path.join(plot_path, filename)
        plot_array_range(array, x_range=(0,incr), plotpath = plot_filepath, binary=True)

In [ ]:
vfrange_plot_path = os.path.join(produced_path, 'vfrange_plots')
os.makedirs(vfrange_plot_path, exist_ok=True)

In [ ]:
# gen_data_dic.keys()

In [ ]:
volfrac_uc_names = ['FinicalCall', 'ImplicitBialy','CelticMaltese','CleanSnout','FitThought', 'MiscibleHeadshot',
                    'PowerfulFeijoa','StormyShock','TerminalMarocain','TolerantKite','UpliftedAeolian',
                    'ZigzagLogomach','AuralAlexic',]

In [ ]:
source_dic = loaded_dict

In [ ]:
# This for producing plots of all the volume fractions for a given topo

for topo in volfrac_uc_names:
    topodic = source_dic[topo]
    
    for key in list(topodic.keys())[1:]:
        vf_tgt = str(topodic[key]['Target volume fraction'])[:5]
        vf_actual = topodic[key]['Actual volume fraction']['string'][:5]
        
        for arrtype in ['decoded array',]:#'binary array']:
            if arrtype.split(' ')[0] == 'binary':
                binary=True
                suffix='bin'
            else:
                binary=False
                suffix='cont'
                
            filename = f'{topo}_tgt{vf_tgt}_actual{vf_actual}_{suffix}'
            
            array = topodic[key][arrtype]
            
            plot_range = (32, 64)
            
            for incr in plot_range:
                if incr==32:
                    plotincr = 'half'                
                else:
                    plotincr = 'full'
                incr_filename = filename + '_' + plotincr
                plotpath = os.path.join(vfrange_plot_path, incr_filename)
                
                plot_array_range(array, x_range=(0,incr), plotpath = plotpath, binary=binary)

In [ ]:
# pngpath

In [ ]:
# from scipy.io import loadmat
# import os

In [ ]:
# MLTO_2023/10_VAE_regressor/vae_produced_UCs/MAT_files/

In [ ]:
# os.path.join(mat_path, 'BandagedPatrial_mult_neg_0.1_vf0.831.mat')

In [ ]:
# loadmat(os.path.join(mat_path, 'BandagedPatrial_mult_neg_0.1_vf0.831.mat'))